# 调包

In [1]:
import tushare as ts
import pandas as pd
import numpy as np
from flask import Flask
import string
from datetime import datetime

In [2]:
app = Flask(__name__)

# 接入API

In [3]:
token='a9e59626e55bb3444009b846bc03be03b91eaaf87ee49c9a95b74ef6'
ts.set_token(token)
pro = ts.pro_api()

# 目标1

用户给定ts_code,返回：①ts_code错误②ts_code正确

# is_ts_code()

判断用户输入的ts_code是否正确

In [4]:
def is_ts_code(ts_code):
    '''
    查询字符串是否符合：XX.XX的格式
    获取.后的字符
    查询字符是否在交易所列表中
    通过字典方法获得交易所全称
    通过fut_basic获得交易所所有期货的ts_code列表
    查询ts_code是否在该列表中
    '''
    is_index = 0
    if '.' in ts_code:
        exchange = ts_code.split('.')[-1]
        if exchange in ['CFX','DCE','ZCE','SHF','INE','GFE']:
            map_dic = dict(zip(['CFX','DCE','ZCE','SHF','INE','GFE'],['CFFEX','DCE','CZCE','SHFE','INE','GFEX']))
            exchange_real = map_dic[exchange]
            ts_code_lst = list(pro.fut_basic(exchange=exchange_real, fut_type='1', fields='ts_code')['ts_code'])
            if ts_code in ts_code_lst:
                is_index = 1
    return is_index

In [5]:
is_ts_code('CU1811SHF')

0

In [6]:
is_ts_code('CU1811.SHF')

1

In [7]:
@app.route('/<ts_code>')
def goal_1(ts_code):
    '''
    查询字符串是否符合：XX.XX的格式
    获取.后的字符
    查询字符是否在交易所列表中
    通过字典方法获得交易所全称
    通过fut_basic获得交易所所有期货的ts_code列表
    查询ts_code是否在该列表中
    '''
    is_ts_code_index = is_ts_code(ts_code)
    if is_ts_code_index == 1:
        return 'ts_code正确'
    else:
        return 'ts_code错误'
       

# 目标2

用户给定ts_code和时间，返回：①ts_code错误 ②ts_code正确但是时间不正确 ③ts_code正确且时间正确，返回数据

# is_time():

用户输入时间格式是否符合年份月份日期格式以及是否满足基本法（比如日期为32一定错了，闰年二月可以有29号）

In [8]:
def time_tran(time):
        '''
        把Ymd转化为Y-m-d，方便后续处理
        '''
        year = time[0:4]
        month = time[4:6]
        date = time[6:]
        date_str = year + '-' + month + '-' + date
        return date_str

In [9]:
def is_time(time):
    is_index = 0
    if len(time) == 8:
        date_str = time_tran(time)
        try:
            date_object = datetime.strptime(date_str, '%Y-%m-%d')
        except ValueError:
            pass
        else:
            is_index = 1
    return is_index

In [10]:
is_time('2018201')

0

In [11]:
is_time('20180231')

0

In [12]:
is_time('20180201')

1

In [13]:
def is_heyue(ts_code,start_time,end_time):
    exchange = ts_code.split('.')[-1]
    map_dic = dict(zip(['CFX','DCE','ZCE','SHF','INE','GFE'],['CFFEX','DCE','CZCE','SHFE','INE','GFEX']))
    exchange_real = map_dic[exchange]
    df = pro.fut_basic(exchange=exchange_real, fut_type='1', fields='ts_code,list_date,delist_date')
    st = list(df[df['ts_code']==ts_code]['list_date'])[0]
    ed = list(df[df['ts_code']==ts_code]['delist_date'])[0]
    #判断是否在合约期间
    st_1 = datetime.strptime(time_tran(start_time),'%Y-%m-%d')
    st_2 = datetime.strptime(time_tran(st),'%Y-%m-%d')
    ed_1 = datetime.strptime(time_tran(end_time),'%Y-%m-%d')
    ed_2 = datetime.strptime(time_tran(ed),'%Y-%m-%d')
    if (st_1 >= st_2) and (ed_1 <= ed_2):
        return 1

In [14]:
@app.route('/<ts_code>/st=<start_time>ed=<end_time>')
def goal_2(ts_code,start_time,end_time):
    '''
    ts_code是否正确
    st和ed格式是否正确
    获取ts_code合约时间
    查询st和ed是否在该范围内
    返回所有时间
    '''
    is_ts_index = is_ts_code(ts_code)
    is_st_index = is_time(start_time)
    is_ed_index = is_time(end_time)
    if is_ts_index:
        if is_st_index and is_ed_index:
            if is_heyue(ts_code,start_time,end_time):
                result = pro.fut_daily(ts_code= ts_code , start_date = start_time, end_date = end_time)
                return result.to_json(orient="records", force_ascii=False)
            else:
                return '输入时间段不在合约期间'
        else:
            return '时间格式错误'
    else:
        return 'ts_code错误'
    

# 目标3

用户给定ts_code和时间和index(high,low,close,open),返回：①ts_code错误 ②ts_code正确但是时间不正确 ③ts_code正确且时间正确，返回数据

In [15]:
@app.route('/<ts_code>/st=<start_time>ed=<end_time>/<key>')
def goal_3(ts_code,start_time,end_time,key):
    is_ts_index = is_ts_code(ts_code)
    is_st_index = is_time(start_time)
    is_ed_index = is_time(end_time)
    if is_ts_index:
        if is_st_index and is_ed_index:
            if is_heyue(ts_code,start_time,end_time):
                if key in ['open','high','low','close']:
                    result = pro.fut_daily(ts_code= ts_code , start_date = start_time, end_date = end_time)[['trade_date',key]]
                    return result.to_json(orient="records", force_ascii=False)
                else:
                    return '关键词必须为：open high low close'
            else:
                return '输入时间段不在合约期间'
        else:
            return '时间格式错误'
    else:
        return 'ts_code错误'

In [ ]:
if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off
2023-03-12 01:38:40,829 - werkzeug - INFO - WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
2023-03-12 01:38:40,829 - werkzeug - INFO - Press CTRL+C to quit
